In [59]:
import numpy as np
import matplotlib.pyplot as plt
import gc
import cPickle as pickle
import sys
sys.path.append('../../mainstream/src/scheduler')
sys.path.append('../../mainstream/src/scheduler/types')
sys.path.append('../../mainstream/data')
import scheduler_util
from collections import defaultdict
%matplotlib inline

In [60]:
stats = []
gc.disable()
for i in range(4):
    try:
        with open('../../mainstream/data/graph-b100-{}.pkl'.format(i), 'rb') as f:
            stats.append(pickle.load(f))
    except Exception:
        pass
gc.enable()

In [61]:
all_edges = set()
for dp, edges in stats[1:]:
    all_edges |= edges
    print(len(edges), len(all_edges))

(993, 993)
(5919, 6912)
(11908, 18820)


In [62]:
nodes = set()
for a, b in all_edges:
    nodes.add(a)
    nodes.add(b)

In [70]:
def id_stem(stem_):
    if stem_ is None:
        return 'root'
    if not isinstance(stem_, tuple):
        stem_ = stem_.stem
    assert len(stem_) == len(set(x[0] for x in stem_))
    return '{}_{}-{}'.format(len(stem_), len(set(x[1] for x in stem_)), hex(hash((4131,) + stem_ + (4321,123)))[-16:])

node_ids = {}
for node in nodes:
    if id_stem(node) in node_ids:
        assert node.stem == node_ids[id_stem(node)].stem 
#         print node.stem, node_ids[id_stem(node)].stem
    node_ids[id_stem(node)] = node
# assert len(nodes) == len(node_ids), (len(nodes), len(node_ids))
unique_edges = set((a.stem, b.stem) for a, b in all_edges)
for a in stats[0][0]:
    unique_edges.add((None, a.stem))

In [71]:
with open('../output/mainstream/stems/edges.csv', 'w') as f:
    for a, b in unique_edges:
        f.write(id_stem(a) + ',' + id_stem(b) + '\n')

In [72]:
weights = defaultdict(int)
for dp, edges in stats:
    for stem, v in dp.iteritems():
        weights[id_stem(stem)] += len(v)

In [73]:
with open('../output/mainstream/stems/nodes.csv', 'w') as f:
    f.write('id,label,weight\n')
    for node in node_ids:
        f.write('{},{},{}\n'.format(node, node, weights[node]))

In [74]:
edge_weights = defaultdict(int)
for dp, edges in stats:
    for stem, v in dp.iteritems():
        for vv in v:
            info = vv[2]
            edge_weights[(id_stem(info['prev_stem']), id_stem(stem))] += 1



In [79]:
with open('../output/mainstream/stems/edges_weighted.csv', 'w') as f:
    f.write('source,target,weight\n')
    for a, b in unique_edges:
        f.write(id_stem(a) + ',' + id_stem(b) + ',' + str(edge_weights[(id_stem(a), id_stem(b))]) + '\n')

with open('../output/mainstream/stems/edges_weighted_scaled.csv', 'w') as f:
    f.write('source,target,weight,weight_original,weight_a,weight_b\n')
    for a, b in unique_edges:
        weight = edge_weights[(id_stem(a), id_stem(b))]
        f.write('{},{},{:g},{},{},{}\n'.format(id_stem(a), id_stem(b), float(weight) / weights[id_stem(b)], weight, weights[id_stem(a)], weights[id_stem(b)]))


In [84]:
from collections import Counter
stem_lens = Counter()
for stem in nodes:
    stem_lens[len(stem.stem)] += 1
stem_lens

Counter({1: 118, 2: 726, 3: 702, 4: 126})